# RNNレイヤの実装

In [2]:
import numpy as np
from common.functions import sigmoid

### [演習]
* 以下のTimeRNNレイヤのクラスを完成させましょう

In [3]:
class RNN:
    def __init__(self, Wx, Wh, b):
        """
        Wx : 入力xにかかる重み
        Wh : １時刻前のhにかかる重み
        b : バイアス
        """
        
        # パラメータのリスト
        self.params = [Wx, Wh, b]
        
        # 勾配のリスト
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        """
        順伝播計算
        """
        Wx, Wh, b = self.params
        
        # 行列の積　+　行列の積 + バイアス
        t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
        
        # 活性化関数に入れる
        h_next = np.tanh(t)

        # 値の一時保存
        self.cache = (x, h_prev, h_next)
        
        return h_next

    def backward(self, dh_next):
        """
        逆伝播計算
        """
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache

        # tanhでの逆伝播
        # dh_next * (1 - y^2)
        A3 = dh_next * (1 - h_next ** 2)
        
        # バイアスbの勾配
        # Nの方向に合計する
        db = np.sum(A3, axis=0)
        
        # 重みWhの勾配
        dWh = np.dot(h_prev.T, A3)
        
        # 1時刻前に渡す勾配
        dh_prev = np.dot(A3, Wh.T)
        
        # 重みWxの勾配
        dWx = np.dot(x.T, A3)
        
        # 入力xに渡す勾配
        dx = np.dot(A3, Wx.T)

        # 勾配をまとめる
        self.grads[0] = dWx
        self.grads[1] = dWh
        self.grads[2] = db

        return dx, dh_prev

In [4]:
D = 1 # 入力層のノード数
H = 5 # 中間層のノード数
Wx = np.random.randn(D, H)
Wh = np.random.randn(H, H)
b = np.zeros(H)

# オブジェクトの生成
rnn = RNN(Wx, Wh, b)

# 順伝播計算
N = 4 # バッチサイズ
x = np.random.randn(N, D)
h_prev = np.random.randn(N, H)
h_next = rnn.forward(x, h_prev)
print("h_next=", h_next)
print()

# 逆伝播計算
dh_next = np.random.randn(N, H )
dx, dh_prev = rnn.backward(dh_next)
print("dx=", dx)
print()
print("dh_prev=", dh_prev)
print()


h_next= [[-0.99652307 -0.946302   -0.99995941 -0.98376948  0.74944177]
 [-0.92473991  0.98440793 -0.11049856 -0.73101109  0.98869462]
 [-0.90770961 -0.09719778 -0.19842126 -0.58797553 -0.04650672]
 [ 0.98906712 -0.99879504 -0.95967091  0.74021008  0.9995459 ]]

dx= [[ 0.7278653 ]
 [-0.23283255]
 [-0.51971531]
 [ 0.14800593]]

dh_prev= [[ 0.34664104  1.2724381  -0.00627001  0.03929235 -0.22217923]
 [ 0.12422263 -0.7705414   0.40740494  0.54429844 -0.06576292]
 [ 0.00393132 -1.00677578  0.62787488  0.30069188  0.42819771]
 [ 0.0438729  -0.22252617  0.02286547  0.17761975 -0.05564238]]

